## CheXpert Dataset Classification with ResNet50, EfficientNet

In [1]:
###
import pandas as pd
import numpy as np
import os
import cv2
import warnings
warnings.filterwarnings("ignore")
# import matplotlib.pyplot as plt
import datetime
from pathlib import Path
import sys
import time
####
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
####
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch import optim
import torchvision.transforms as transforms
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision import models
from torch.autograd import Variable
from torchtoolbox.tools import mixup_data, mixup_criterion
##
from fastprogress import master_bar, progress_bar
from PIL import Image
from tqdm import tqdm
import copy

In [2]:
BASE_DIR=os.getcwd()+'/'
print(BASE_DIR)

/home/csl/jhb/


### Train, Valid, Test Data

In [3]:
train_csv=BASE_DIR+'CheXpert-v1.0/train.csv'
valid_csv=BASE_DIR+'CheXpert-v1.0/valid.csv'
test_csv=BASE_DIR+'test_labels.csv'
train_df=pd.read_csv(train_csv)
valid_df=pd.read_csv(valid_csv)
test_df=pd.read_csv(test_csv)

In [9]:
print(" Train data:",len(train_df))
train_df.head()

 Train data: 223414


,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0/train/patient00001/study1/view1_...,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0
1,CheXpert-v1.0/train/patient00002/study2/view1_...,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN
2,CheXpert-v1.0/train/patient00002/study1/view1_...,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
3,CheXpert-v1.0/train/patient00002/study1/view2_...,Female,83,Lateral,NaN,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,CheXpert-v1.0/train/patient00003/study1/view1_...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [10]:
print(" Validation data:",len(valid_df))
valid_df.head()

 Validation data: 234


,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0/valid/patient64541/study1/view1_...,Male,73,Frontal,AP,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,CheXpert-v1.0/valid/patient64542/study1/view1_...,Male,70,Frontal,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,CheXpert-v1.0/valid/patient64542/study1/view2_...,Male,70,Lateral,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,CheXpert-v1.0/valid/patient64543/study1/view1_...,Male,85,Frontal,AP,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CheXpert-v1.0/valid/patient64544/study1/view1_...,Female,42,Frontal,AP,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
print(" Test data:",len(test_df))
test_df.head()

 Test data: 668


,Path,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,test/patient64741/study1/view1_frontal.jpg,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,test/patient64742/study1/view1_frontal.jpg,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,test/patient64743/study1/view1_frontal.jpg,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,test/patient64744/study1/view1_frontal.jpg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,test/patient64744/study1/view2_lateral.jpg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


### Training Data Analyze

In [12]:
label_cols=train_df.columns[5:]
print("Labels:",label_cols)

Labels: Index(['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly',
       'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia',
       'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other',
       'Fracture', 'Support Devices'],
      dtype='object')


In [33]:
# Check data distribution
# each column label, i will check how many Nan , -1, 0 and 1
train_distribution_df=pd.DataFrame()
valid_distribution_df=pd.DataFrame()
test_distribution_df=pd.DataFrame()

train_distribution_df['label']=label_cols
valid_distribution_df['label']=label_cols
test_distribution_df['label']=label_cols

train_distribution_df['Nan']=[train_df[col].isnull().sum() for col in label_cols]
train_distribution_df['-1']=[train_df[train_df[col]==-1].shape[0] for col in label_cols]
train_distribution_df['0']=[train_df[train_df[col]==0].shape[0] for col in label_cols]
train_distribution_df['1']=[train_df[train_df[col]==1].shape[0] for col in label_cols]

valid_distribution_df['Nan']=[valid_df[col].isnull().sum() for col in label_cols]
valid_distribution_df['-1']=[valid_df[valid_df[col]==-1].shape[0] for col in label_cols]
valid_distribution_df['0']=[valid_df[valid_df[col]==0].shape[0] for col in label_cols]
valid_distribution_df['1']=[valid_df[valid_df[col]==1].shape[0] for col in label_cols]

test_distribution_df['Nan']=[test_df[col].isnull().sum() for col in label_cols]
test_distribution_df['-1']=[test_df[test_df[col]==-1].shape[0] for col in label_cols]
test_distribution_df['0']=[test_df[test_df[col]==0].shape[0] for col in label_cols]
test_distribution_df['1']=[test_df[test_df[col]==1].shape[0] for col in label_cols]

print("Train data distribution")
print(train_distribution_df)


Train data distribution
                         label     Nan     -1      0       1
0                   No Finding  201033      0      0   22381
1   Enlarged Cardiomediastinum  178575  12403  21638   10798
2                 Cardiomegaly  177211   8087  11116   27000
3                 Lung Opacity  105636   5598   6599  105581
4                  Lung Lesion  211470   1488   1270    9186
5                        Edema  137458  12984  20726   52246
6                Consolidation  152792  27742  28097   14783
7                    Pneumonia  195806  18770   2799    6039
8                  Atelectasis  154971  33739   1328   33376
9                 Pneumothorax  144480   3145  56341   19448
10            Pleural Effusion   90203  11628  35396   86187
11               Pleural Other  216922   2653    316    3523
12                    Fracture  211220    642   2512    9040
13             Support Devices  100197   1079   6137  116001


In [35]:
print("Validation data distribution")
print(valid_distribution_df)


Validation data distribution
                         label  Nan  -1    0    1
0                   No Finding    0   0  196   38
1   Enlarged Cardiomediastinum    0   0  125  109
2                 Cardiomegaly    0   0  166   68
3                 Lung Opacity    0   0  108  126
4                  Lung Lesion    0   0  233    1
5                        Edema    0   0  189   45
6                Consolidation    0   0  201   33
7                    Pneumonia    0   0  226    8
8                  Atelectasis    0   0  154   80
9                 Pneumothorax    0   0  226    8
10            Pleural Effusion    0   0  167   67
11               Pleural Other    0   0  233    1
12                    Fracture    0   0  234    0
13             Support Devices    0   0  127  107


In [36]:
print("Test data distribution")
print(test_distribution_df)

Test data distribution
                         label  Nan  -1    0    1
0                   No Finding    0   0  559  109
1   Enlarged Cardiomediastinum    0   0  370  298
2                 Cardiomegaly    0   0  493  175
3                 Lung Opacity    0   0  358  310
4                  Lung Lesion    0   0  654   14
5                        Edema    0   0  583   85
6                Consolidation    0   0  633   35
7                    Pneumonia    0   0  654   14
8                  Atelectasis    0   0  490  178
9                 Pneumothorax    0   0  658   10
10            Pleural Effusion    0   0  548  120
11               Pleural Other    0   0  660    8
12                    Fracture    0   0  662    6
13             Support Devices    0   0  353  315


## Model Select

In [3]:
import timm

timm.list_models('seresnet*')

['seresnet18',
 'seresnet33ts',
 'seresnet34',
 'seresnet50',
 'seresnet50t',
 'seresnet101',
 'seresnet152',
 'seresnet152d',
 'seresnet200d',
 'seresnet269d',
 'seresnetaa50d']